In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path

%load_ext autoreload
%autoreload 2
from base import datahandler, prediction_models, evaluation

2024-01-07 19:47:13.426823: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-07 19:47:13.426903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-07 19:47:13.496382: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-07 19:47:13.645276: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-07 19:47:14.975076: W tensorflow/compiler/tf2

In [2]:
ew_localizer = tf.keras.models.load_model("models/ew_localizer.hdf5")
ns_localizer = tf.keras.models.load_model("models/ns_localizer.hdf5")
ew_ns_classifier = tf.keras.models.load_model("models/ew_ns_classifier.hdf5")

2024-01-07 19:47:17.697211: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-07 19:47:17.948321: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-07 19:47:17.948405: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-07 19:47:17.950120: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-07 19:47:17.950193: I external/local_xla/xla/stream_executor

In [3]:
# load some validation data
challenge_data_dir = Path('dataset/phase_1/')
data_dir = challenge_data_dir / "train"
labels_dir = challenge_data_dir / 'train_labels.csv'

split_dataframes = datahandler.load_and_prepare_dataframes(data_dir, labels_dir)
train_keys = list(split_dataframes.keys())[150:]
val_keys = list(split_dataframes.keys())[150:]
some_dataframes = {df_k : split_dataframes[df_k] for df_k in val_keys}

input_features_reduced = ['Eccentricity', 'Semimajor Axis (m)', 'Inclination (deg)', 'RAAN (deg)',
       'Argument of Periapsis (deg)', 'True Anomaly (deg)', 'Latitude (deg)',
       'Longitude (deg)', 'Altitude (m)']

In [16]:
# start with localization
ds_gen = datahandler.DatasetGenerator(split_df=some_dataframes,
                                      input_features=input_features_reduced,
                                      label_features=['EW_Node_Location', 'NS_Node_Location'],
                                      train_val_split=1.0,
                                      stride=1,
                                      input_stride=1,
                                      padding=False,
                                      input_history_steps=40,
                                      input_future_steps=40,
                                      seed=69)
test_ds = ds_gen.get_datasets(256, label_features=['EW_Node_Location', 'NS_Node_Location'], shuffle=False, keep_identifier=True)

inputs = np.concatenate([element for element in test_ds.map(lambda x,y,z: x).as_numpy_iterator()])
identifiers = np.concatenate([element for element in test_ds.map(lambda x,y,z: z).as_numpy_iterator()])

# get predictions
preds_ew = ew_localizer.predict(inputs)
preds_ew_argmax = np.argmax(preds_ew, axis=1)
preds_ns = ns_localizer.predict(inputs)
preds_ns_argmax = np.argmax(preds_ns, axis=1)

df = pd.DataFrame(np.concatenate([identifiers.reshape(-1,2)], axis=1), columns=['ObjectID', 'TimeIndex'], dtype=np.int32)
df[f'Location_EW'] = preds_ew_argmax
df[f'Location_NS'] = preds_ns_argmax

# add initial node prediction
for obj_id in list(df["ObjectID"].unique()):
    df = df.sort_index()
    df.loc[-1] = [int(obj_id), 0, 1, 1] # objid, timeindex, Location_EW, Location_NS
    df.index = df.index + 1
    df = df.sort_index()

df_locs = df.loc[(df['Location_EW'] == 1) | (df['Location_NS'] == 1)]
df_locs = df_locs.sort_values(['ObjectID', 'TimeIndex']).reset_index(drop=True)
print(len(df_locs))
print(df_locs.head(20))

=========================Creating Dataset=========================
Seed: 69
nTrain: 20 nVal: 0 (1.00)
Padding: False
Scaling: True
Horizons: 40-40 @ stride 1
Sin-Transformed features: ['True Anomaly (deg)', 'Argument of Periapsis (deg)']
Labels: ['EW_Node_Location', 'NS_Node_Location']
=========================Finished Dataset=========================
1312/1312 [==============================] - 3s 3ms/step
24
    ObjectID  TimeIndex  Location_EW  Location_NS
0         28          0            1            1
1         59          0            1            1
2         63          0            1            1
3         67          0            1            1
4         86          0            1            1
5         97          0            1            1
6        111          0            1            1
7        111        442            1            0
8        123          0            1            1
9        124          0            1            1
10       124       1725            1

In [23]:
# now we need to get label predictions on those locations
ds_gen = datahandler.DatasetGenerator(split_df=some_dataframes,
                                      input_features=input_features_reduced,
                                      label_features=['EW', 'NS'],
                                      train_val_split=1.0,
                                      stride=1,
                                      input_stride=2, #!
                                      padding=True,
                                      input_history_steps=40,
                                      input_future_steps=40,
                                      seed=69)
test_ds = ds_gen.get_datasets(256, label_features=['EW', 'NS'], shuffle=False, keep_identifier=True)

inputs = np.concatenate([element for element in test_ds.map(lambda x,y,z: x).as_numpy_iterator()])
#labels = np.concatenate([element['EW_Node_Location'] for element in ds.map(lambda x,y,z: y).as_numpy_iterator()])
identifiers = np.concatenate([element for element in test_ds.map(lambda x,y,z: z).as_numpy_iterator()])


# get predictions
preds = ew_ns_classifier.predict(inputs)
sub_dfs = []
for ft_idx, feature in enumerate(['EW', 'NS']):
    sub_df = pd.DataFrame(np.concatenate([identifiers.reshape(-1,2)], axis=1), columns=['ObjectID', 'TimeIndex'], dtype=np.int32)
    sub_df['Direction'] = feature
    preds_argmax = np.argmax(preds[ft_idx], axis=1)
    sub_df[f'{feature}_Decoded'] = ds_gen.combined_label_encoder.inverse_transform(preds_argmax)
    sub_df[['Node', 'Type']] = sub_df[f'{feature}_Decoded'].str.split('-', expand=True)
    sub_df = sub_df.drop([f'{feature}_Decoded'], axis='columns')
    sub_dfs.append(sub_df)

# Add direction column
df_classes = pd.concat(sub_dfs).sort_values(['ObjectID', 'TimeIndex']).reset_index(drop=True)
# For timeindex 0, the node is always SS
df_classes.loc[df_classes['TimeIndex'] == 0, 'Node'] = 'SS'

print(df_classes.head(5))

=========================Creating Dataset=========================
Seed: 69
nTrain: 20 nVal: 0 (1.00)
Padding: True
Scaling: True
Horizons: 40-40 @ stride 2
Sin-Transformed features: ['True Anomaly (deg)', 'Argument of Periapsis (deg)']
Labels: ['EW_encoded', 'NS_encoded']
=========================Finished Dataset=========================
1361/1361 [==============================] - 7s 5ms/step
   ObjectID  TimeIndex Direction Node Type
0        28          0        EW   SS   CK
1        28          0        NS   SS   CK
2        28          1        EW   SS   CK
3        28          1        NS   SS   CK
4        28          2        EW   SS   CK


In [31]:
# Combine the classifications with the node locations
df_merged = df_locs.merge(df_classes, how='left', on = ['ObjectID', 'TimeIndex'])
df_reduced = df_merged[((df_merged['Location_EW'] == 1) & (df_merged['Direction'] == 'EW') | (df_merged['Location_NS'] == 1) & (df_merged['Direction'] == 'NS'))]
df_reduced = df_reduced.drop(['Location_EW'], axis='columns')
df_reduced = df_reduced.drop(['Location_NS'], axis='columns')
print(df_reduced.head(5))

[28, 59, 63, 67, 86, 97, 111, 123, 124, 151, 178, 197, 205, 208, 210, 226, 228, 238, 243, 246]
[28, 59, 63, 67, 86, 97, 111, 123, 124, 151, 178, 197, 205, 208, 210, 226, 228, 238, 243, 246]
   ObjectID  TimeIndex Direction Node Type
0        28          0        EW   SS   CK
1        28          0        NS   SS   CK
2        59          0        EW   SS   CK
3        59          0        NS   SS   CK
4        63          0        EW   SS   CK


In [33]:
# Finally, perform the evaluation
ground_truth = pd.read_csv(challenge_data_dir / 'train_labels.csv')
ground_truth = ground_truth[ground_truth['ObjectID'].isin(list(df_reduced["ObjectID"].unique()))].copy()

evaluator = evaluation.NodeDetectionEvaluator(ground_truth, df_reduced)
precision, recall, f2, rmse, total_tp, total_fp, total_fn = evaluator.score()
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F2: {f2:.2f}')
print(f'RMSE: {rmse:.4}')
print(f'TP: {total_tp} FP: {total_fp} FN: {total_fn}')

Precision: 0.77
Recall: 0.40
F2: 0.45
RMSE: 0.0
TP: 34 FP: 10 FN: 50
